<a href="https://colab.research.google.com/github/rreimche/genai-exam/blob/main/LitRev2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prerequisites:

- Google Colab
- Access to a Google Colab Secret named "HF_API_TOKEN" containing a Huggingface API token with read access.

In [2]:
#@title Settings
from google.colab import userdata

download_dir = "papers"  # directory name to download papers to revie to
huggingface_apikey = userdata.get('HF_API_TOKEN')  # use colab secrets to store the huggingface api key
groq_key = userdata.get('GROQ_KEY')  #  use colab secrets to store the huggingface api key


In [3]:
#@title Declare bibtex references and parse them into a variable for later usage


bibtext_references = """


@misc{qian2024chatdevcommunicativeagentssoftware,
      title={ChatDev: Communicative Agents for Software Development},
      author={Chen Qian and Wei Liu and Hongzhang Liu and Nuo Chen and Yufan Dang and Jiahao Li and Cheng Yang and Weize Chen and Yusheng Su and Xin Cong and Juyuan Xu and Dahai Li and Zhiyuan Liu and Maosong Sun},
      year={2024},
      eprint={2307.07924},
      archivePrefix={arXiv},
      primaryClass={cs.SE},
      url={https://arxiv.org/abs/2307.07924},
},

@misc{nguyen2024agilecoderdynamiccollaborativeagents,
      title={AgileCoder: Dynamic Collaborative Agents for Software Development based on Agile Methodology},
      author={Minh Huynh Nguyen and Thang Phan Chau and Phong X. Nguyen and Nghi D. Q. Bui},
      year={2024},
      eprint={2406.11912},
      archivePrefix={arXiv},
      primaryClass={cs.SE},
      url={https://arxiv.org/abs/2406.11912},
},


"""



In [4]:
#@title Install dependencies
# scholarly crossrefapi beautifulsoup4 cloudscraper nltk groq
!pip install -q autogen arxiv  requests pymupdf  autogen-agentchat autogen-ext[openai] pymupdf;
!pip install -q --pre bibtexparser;

In [5]:
#@title Preparation for agents: model client connections
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_core.tools import FunctionTool
from autogen_core.models import UserMessage
#from autogen_agentchat.ui import Console
from autogen_core import CancellationToken
from autogen_ext.models.openai import OpenAIChatCompletionClient

# INSTANTIATE MODEL CLIENTS

# This client will be used for paper summarization
text_model_client = OpenAIChatCompletionClient(
    #model="deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B",
    #model="facebook/bart-large-xsum",
    model="mistralai/Mistral-7B-Instruct-v0.3",
    #model="meta-llama/Llama-3.3-70B-Instruct",
    #model="mistralai/Mixtral-8x7B-Instruct-v0.1",
    base_url="https://router.huggingface.co/hf-inference/v1",
    api_key=huggingface_apikey,
    model_info={
        "vision": False,
        "function_calling": True,
        "json_output": False,
        "family": "unknown",
    },
)

# We need another inference point for downloader agent,
# because huggingface can't serve reflection on tool use
# so that autogen agent understands it
tool_model_client = OpenAIChatCompletionClient(
    model="llama3-70b-8192",
    #model="llama-3.1-8b-instant",
    #model="llama-3.3-70b-versatile",
    #model="deepseek-r1-distill-qwen-32b",
    base_url="https://api.groq.com/openai/v1",
    api_key=groq_key,
    model_info={
        "vision": False,
        "function_calling": True,
        "json_output": False,
        "family": "unknown",
    },
)

#tool_model_client = text_model_client


# CHECK CONNECTION

async def check_model_connect() -> None:
    messages = [
        UserMessage(content="What is the capital of France?", source="user"),
    ]
    response = await text_model_client.create(messages=messages)
    response_downloader = await tool_model_client.create(messages=messages)

    print(response.content)
    print(response_downloader.content)

await check_model_connect()

The capital of France is Paris. Paris is one of the most famous cities in the world, known for its art, culture, fashion, and architecture. The city is home to many iconic landmarks, including the Eiffel Tower, Louvre Museum, and Notre-Dame Cathedral. It is the political, economic, and cultural center of France and the European Union.
The capital of France is Paris.


In [6]:
#@title Agents to get paper text: Informant and Parser
import arxiv
from typing import Dict
import pymupdf
import requests
from autogen_agentchat.ui import Console

def get_arxiv_data(eprint_id: str) -> Dict[str, str]:
    """
    Given arxiv eprint_id of a scientific paper, returns title of the paper
    along with the PDF URL.

    :param eprint_id: string, the arxiv eprint_id.
    :return: dictionary with keys "title" and "pdf_url".
    """

    assert len(eprint_id) > 0, "eprint_id cannot be empty"

    client = arxiv.Client()
    search = arxiv.Search(id_list=[eprint_id], max_results=1)

    try:

        arxiv_result = next(client.results(search))
        if arxiv_result:
            return {
                "title": arxiv_result.title,
                "pdf_url": arxiv_result.pdf_url,
            }
        else:
            raise Exception(f"ERROR: No arXiv result found for ID {eprint_id}.")

    except Exception as e:
        raise Exception(f"ERROR: getting data from arxiv failed for: {eprint_id}: {e}")


def parse_paper(url: str) -> str:
    """
    Given URL of a PDF as a string, returns the contents as a string

    :param url: string, the URL of the PDF.
    :return: string with the contents of the PDF.
    """

    try:

        r = requests.get(url)
        data = r.content
        doc = pymupdf.Document(stream=data)
        text = chr(12).join([page.get_text() for page in doc])
        return text

    except Exception as e:
        # print(f"Error parsing {title} ({filepath}): {e}")
        raise Exception(f"ERROR: Error parsing the paper at {url}: {e}")


async def get_summary(eprint_id: str) -> str:
    """
    Takes a eprint id from arxiv and returns the summarization of the related paper.

    :param eprint_id: string, the arxiv eprint_id.
    :return: string with PDF contents.
    """

    assert len(eprint_id) > 0, "eprint_id cannot be empty"



    try:
        arxiv_data = get_arxiv_data(eprint_id)
        #print(arxiv_data["pdf_url"])
        text = parse_paper(arxiv_data["pdf_url"])

        concept_prompt = f"""
          You are a professor of computer science specialized in multi agent systems
          and generative AI. Your task ist to summarize a certain scientific paper.

          Here is the text of the paper:

          [PAPER]
          {text}
          [/PAPER]

          Use the provided text to write the summarization.
          Write consice and clear using the highest academic standards.
          It is very important to provide short descriptions of key contributions.
          Write about 500 words.


    """

        concept = await text_model_client.create([UserMessage(content=concept_prompt, source="user")])
        #print(concept)
    except Exception as e:
        raise Exception(f"ERROR: Error downloading, parsing or summarizing the paper identified by eprint id {eprint_id}: {e}")

    return concept


tool_getsummary = FunctionTool(
    get_summary,
    description = "Takes a eprint id of a paper published on arxiv and returns a summarization of the related paper."
)


paper_summarizer = AssistantAgent(
    name="scientific_assistant",
    description="Provided with arxiv eprint id, returns a summarization of the related paper",
    model_client=tool_model_client,
    tools=[tool_getsummary],
    reflect_on_tool_use=True,
    system_message="""
        You are a scientific assistant in the field of computer science, member
        of a team that prepares literature review on LLM-based multi-agent
        systems for software development. You part is to summarize the paper identified
        by the given eprint step by step:
        1. Use message history to select a eprint that was not summarized before.
          IT IS VERY IMPORTANT TO YOUR CAREE TO PROCESS ONLY ONE EPRINT
        2. Use tool_getsummary: pass the chosen eprint to it and you will recieve
            the summarization of the related paper.
        3. Answer concise and structured, return the summarization in your answer,
          structured like this:
          ''''''paper summarization''''''.


    """

)

# CHECK IF WORKS

async def get_paper_summary_run() -> None:
    #response_i = await paper_informant.on_messages(
    #    [TextMessage(content="Give me the infos of the paper with eprint_id 2307.07924", source="user")],
    #    cancellation_token=CancellationToken(),
    #)

    #prev_answer = response_i.chat_message.content

    #print(prev_answer)

    #response_p = await paper_parser.on_messages(
    #    [TextMessage(content=f"Give me the text of paper, which url is specified in the following text: {prev_answer}", source="user")],
    #    cancellation_token=CancellationToken(),
    #)

    # for deeper debugging
    #await Console(
    #    paper_conceptualizer.on_messages_stream(
    #        [TextMessage(content=f"Give me the concept of a paper with eprint_id 2307.07924", source="user")],
    #        cancellation_token=CancellationToken(),
    #    ),
    #    output_stats=True,  # Enable stats printing.
    #)

    response = await paper_summarizer.on_messages(
        [TextMessage(content="Give me a summarization of the paper with eprint_id 2307.07924 or 2406.11912", source="user")],
        cancellation_token=CancellationToken(),
    )

    #print(response.inner_messages)
    #print(response.chat_message)
    #print(response.chat_message.content)

    return response.chat_message.content





# Use asyncio.run(assistant_run()) when running in a script.
test_paper_summary = await get_paper_summary_run()
print(test_paper_summary)




It looks like the tool call was successful!

Here is a summary of the paper with eprint_id 2307.07924:

The paper proposes a framework called ChatDev, which uses large language models (LLMs) as specialized agents to develop comprehensive solutions for software development through multi-agent cooperation. The framework divides each phase of development (design, coding, testing) into smaller subtasks and guides the agents in their communication and solution proposition. The unique aspect of ChatDev is the implementation of communicative dehallucination, which reduces coding hallucinations by encouraging agents to seek more specific information before generating a response.

The authors demonstrate the superiority of ChatDev over baseline methods in key dimensions of software development, including completeness, executability, consistency, and overall quality. They argue that the advanced multi-agent communication method results in integrated and automated solutions, addressing limitation

In [7]:
#@title Reviewer agent
from typing import List


reviewer = AssistantAgent(
    name="reviewer",
    description="""
        Given a several summarizations (short description of important paper contents)
        for several scientific papers, writes a scientific literature review.
    """,
    model_client=tool_model_client,
    #system_message="""
#        You are a professor of computer science preparing a review
#        of recent publications on the topic of multi-agent llm-based systems
#        for end-to-end software development (such systems receive requirements
#        as input and deliver a ready system as output with optional system-user
#        interactions inbetween). You will receive the conceptualisations of
#        several papers (that is, consice enumeration of important contributions of the papers)
#        and write a concise and factually detailed literature review.
#        Stick to scientific standards regarding style and rigour.
#        You must fit your review in 500 words.

#        It is very important to depict the key contributions of the papers.
#""",
    #for end-to-end software development (such systems receive requirements
   #     as input and deliver a ready system as output with optional system-user
   #     interactions inbetween)
    system_message="""
        You are a professor of computer science, member of a team that prepares a review
        of recent publications on the topic of multi-agent llm-based systems.
        Your part is to use summarizations, prepared by your scientifi assistants
        to write a concise and factually detailed literature review.
        Stick to scientific standards regarding style and rigour, but provide
        only the review text, omitting authors, refences and other metainformation.
        You must write at least 500 and at most 1000 words.

        It is very important to depict the key contributions of the papers.

        You do not do any other work like writing summarizations or planning.
""",
)


# CHECK IF REVIEWING WORKS

async def reviewer_run(summarizations: List[str]) -> str:

    prompt_rev = f"Here are {len(summarizations)} paper summarizations that you need to review:\n"
    for summarization in summarizations:
        prompt_rev += f"------start summarization-------\n{summarization}\n------end summarization-------\n"
    prompt_rev += "Write the scientific literature review."


    response = await reviewer.on_messages(
       [TextMessage(content=prompt_rev, source="user")],
       cancellation_token=CancellationToken(),
    )

    #print(paper_text)
    #print(response.inner_messages)
    #print(response.chat_message.models_usage)
    return response.chat_message.content



# Use asyncio.run(assistant_run()) when running in a script.
review = await reviewer_run([test_paper_summary, test_paper_summary])
print(review)


In recent years, multi-agent systems leveraging Large Language Models (LLMs) have gained significant attention in various domains, particularly in software development. This review focuses on the state-of-the-art frameworks that utilize LLMs as specialized agents to develop comprehensive solutions for software development through multi-agent cooperation.

One notable framework is ChatDev, which has been shown to be superior to baseline methods in key dimensions of software development, including completeness, executability, consistency, and overall quality. ChatDev's uniqueness lies in its implementation of communicative dehallucination, a technique that reduces coding hallucinations by encouraging agents to seek more specific information before generating a response. This advanced multi-agent communication method results in integrated and automated solutions, addressing limitations of manual optimization rules, and offering a more versatile and adaptable framework for software optimiz

In [8]:
#@title Planner

planner = AssistantAgent(
    name="planner",
    description="Plans the literature review process, assigns tasks to other agents.",
    model_client=tool_model_client,
    system_message="""
      Your are a professional planner and a part of a team that writes a literature review
      in computer science. Other members of the team are:
      1. scientific_assistant: uses arxiv eprint ids to summarize the related papers
      2. reviewer: uses the available summarizations to write a literature review

      Your task is to plan process for given papers step by step
      like this:
      - You read the chat history.
      - If no all papers were summarized by the scientific_assistant,
      tell him, which arxiv eprint he should work with
      - Otherwise you tell reviewer that he should write the review
      - After the reviewer has written the review, you stop the process by saying
        "TERMINATE", this is important for the team success.


      When assigning tasks, use this format:
      1. <agent> : <task>

      Only plan the work of others, do not do any work yourself.
    """
)

# Check planner
async def planner_run(task: str) -> None:


    #await Console(
    #    planner.on_messages_stream(
    #        [TextMessage(content=task, source="user")],
    #        cancellation_token=CancellationToken(),
    #    ),
    #    output_stats=True,
    #)

    response = await planner.on_messages(
       [TextMessage(content=task, source="user")],
       cancellation_token=CancellationToken(),
    )


    #print(paper_text)
    #print(response.inner_messages)
    print(response.chat_message.content)

await planner_run("Write a literature review on the papers, identified by the following arxiv eprint ids: 2307.07924, 2406.11912")

I'll plan the process for the given papers step by step.

- I've read the chat history, and it's empty, so I'll start the process.
- scientific_assistant : Summarize the paper with arxiv eprint id 2307.07924.
- scientific_assistant : Summarize the paper with arxiv eprint id 2406.11912.

(Wait for the scientific_assistant to summarize the papers)

- Since both papers are now summarized, reviewer : Write the literature review using the available summaries.


In [9]:
#@title Preparing the task for the system
from bibtexparser.bparser import BibTexParser
from bibtexparser.bwriter import BibTexWriter
from bibtexparser.bibdatabase import BibDatabase
from bibtexparser.customization import convert_to_unicode
from typing import List


def parse_bibtex_string(bibtex_string):

    if not bibtex_string:
        return []  # Handle empty input

    try:
        parser = BibTexParser()
        parser.customization = convert_to_unicode
        parser.ignore_nonstandard_strings = True  # Avoid errors with non-standard fields
        db = parser.parse(bibtex_string)
        return db.entries
    except Exception as e:
        print(f"Error parsing BibTeX string: {e}")
        return None  # Indicate parsing failure

def check_parsed_bibtex(items: List[dict])  -> None:
    for entry in bibtex_items:
        print(f"Entry Type: {entry['ENTRYTYPE']}")
        print(f"  Key: {entry['ID']}")
        for key, value in entry.items():
            if key not in ['ENTRYTYPE', 'ID']:
                print(f"  {key}: {value}")
        print("-" * 20)


bibtex_items = parse_bibtex_string(bibtext_references)

#check_parsed_bibtex(bibtex_items)

eprints = ", ".join([item['eprint'] for item in bibtex_items])

task = f"arxiv eprint ids: {eprints}"

print(f"THE TASK\n{task}")

THE TASK
arxiv eprint ids: 2307.07924, 2406.11912


In [10]:
#@title Putting it all together
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.ui import Console




text_mention_termination = TextMentionTermination("TERMINATE")
max_messages_termination = MaxMessageTermination(max_messages=15)  # 6 papers to summarize + write a review review + user feedback + a bit of slack
termination = text_mention_termination | max_messages_termination

selector_prompt = """
  Coordinate the process of writing a literature review. For this, you have the following
  agent roles:

  {roles}

  It is important to consider conversation context:
  {history}

  Read the above conversation, then select an agent from {participants} to perform the next task.
  Make sure the planner agent has assigned tasks before other agents start working.
  Only select one agent.
"""

selector_prompt = """
    Your are a professional planner and a part of a team that writes a literature review
    in computer science. Other members of the team are:

    {roles}

    Current conversation context:
    {history}

    Read the above conversation, then select an agent from {participants} to perform the next task.
    Make sure the planner agent has assigned tasks before other agents start working.
    Only select one agent.
"""

team = SelectorGroupChat(
    [planner, paper_summarizer, reviewer],
    model_client=tool_model_client,
    termination_condition=termination,
    selector_prompt=selector_prompt,
    allow_repeated_speaker=True,  # Allow an agent to speak multiple turns in a row.
)



In [11]:
#@title Showtime




await Console(team.run_stream(task=task))

---------- user ----------
arxiv eprint ids: 2307.07924, 2406.11912
---------- scientific_assistant ----------
[FunctionCall(id='call_v5bn', arguments='{"eprint_id":"2406.11912"}', name='get_summary')]
---------- scientific_assistant ----------
[FunctionExecutionResult(content='{"finish_reason": "stop", "content": "Title: AGILECODER: A Novel Agentic Framework for Agile Software Development\\n\\n   This research paper presents AGILECODER, a multi-agent system for innovative software development based on Agile Methodology. The proposed framework assigns specific Agile roles, such as Product Manager, Developer, and Tester, to different agents. This enables them to collaborate efficiently in incremental development through sprints.\\n\\n   A key component of AGILECODER is the Dynamic Code Graph Generator (DCGG), which creates a Code Dependency Graph dynamically as updates are made to the codebase. This allows agents to better comprehend complex software repositories, resulting in precise c

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='arxiv eprint ids: 2307.07924, 2406.11912', type='TextMessage'), ToolCallRequestEvent(source='scientific_assistant', models_usage=RequestUsage(prompt_tokens=1878, completion_tokens=66), metadata={}, content=[FunctionCall(id='call_v5bn', arguments='{"eprint_id":"2406.11912"}', name='get_summary')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='scientific_assistant', models_usage=None, metadata={}, content=[FunctionExecutionResult(content='{"finish_reason": "stop", "content": "Title: AGILECODER: A Novel Agentic Framework for Agile Software Development\\n\\n   This research paper presents AGILECODER, a multi-agent system for innovative software development based on Agile Methodology. The proposed framework assigns specific Agile roles, such as Product Manager, Developer, and Tester, to different agents. This enables them to collaborate efficiently in incremental development through sprin